In [ ]:
# Faiss
# Facebook AI Similarity Search (Faiss)는 밀집 벡터의 효율적인 유사도 검색과 클러스터링을 위한 라이브러리입니다.
# Faiss는 RAM에 맞지 않을 수도 있는 벡터 집합을 포함하여 모든 크기의 벡터 집합을 검색하는 알고리즘을 포함하고 있습니다.
# 또한 평가와 매개변수 튜닝을 위한 지원 코드도 포함되어 있습니다.
#
# => ** Faiss에서는 유사도=거리 이므로, 작을수로(0에 가까울수록) 유사도가 높은 것임.
#
# GPU 지원 버전을 사용하려면 faiss-gpu를 설치할 수도 있습니다.
%pip install -U langchain-community faiss-cpu langchain-openai tiktoken


In [ ]:
# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}' 식으로 입력한다.
# API 키를 환경변수로 관리하기 위한 .env설정 파일 로딩
import os
from dotenv import load_dotenv

load_dotenv() # API 키 정보 로드
print(f"[OPENAI_API_KEY]\n{os.environ['OPENAI_API_KEY']}\n")
print(f"[HUGGINGFACEHUB_API_TOKEN]\n{os.environ['HUGGINGFACEHUB_API_TOKEN']}")

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

# FAISS에서 AVX2 최적화를 사용하지 않으려면 다음 줄의 주석을 해제하세요.
# import os
#
# os.environ['FAISS_NO_AVX2'] = '1'

# TextLoader를 사용하여 텍스트 파일을 로드합니다.
loader = TextLoader("../data/08.보안관리규정_11.10.01.txt")

# 로드된 문서를 가져옵니다.
documents = loader.load()

# CharacterTextSplitter를 사용하여 문서를 분할합니다.
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)

# 분할된 문서를 가져옵니다.
docs = text_splitter.split_documents(documents)
print(f'*docs 길이: {len(docs)}')

# OpenAIEmbeddings를 사용하여 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings()

# FAISS를 사용하여 문서와 임베딩으로부터 데이터베이스를 생성합니다.
db = FAISS.from_documents(docs, embeddings)

In [ ]:
# 쿼리(query) 변수에 저장된 질문과 유사한 문서를 데이터베이스에서 검색합니다.
query = "네트워크 관련 보안정책은?"
docs = db.similarity_search(query)  # 질문과 유사한 문서를 데이터베이스에서 검색

print(f"문서의 개수: {len(docs)}")
print("[검색 결과]\n")
for i in range(len(docs)):
    print(docs[i].page_content)
    print("===" * 20)
    

In [ ]:
# Retriever로 활용
# vectorstore를 Retriever 클래스로 변환할 수도 있습니다.
# db.as_retriever() 메서드를 호출하여 데이터베이스를 검색기(retriever)로 사용할 수 있는 객체를 생성합니다.
# 이 메서드는 데이터베이스 객체 db를 검색기로 변환합니다.
# 반환된 retriever 객체는 질의에 대한 관련 문서를 데이터베이스에서 검색하는 데 사용됩니다.

# 데이터베이스를 검색기로 사용하기 위해 retriever 변수에 할당합니다.
retriever = db.as_retriever()
# 검색 질의를 사용하여 관련 문서를 검색합니다.
query = "정보보안감사와 관련된 보안정책은?"
docs = retriever.invoke(query)

print(f"문서의 개수: {len(docs)}")
print("[검색 결과]\n")
for i in range(len(docs)):
    print(docs[i].page_content)
    print("===" * 20)

In [ ]:
# 점수에 기반한 유사도 검색
query = "정보보안감사와 관련된 보안정책은?"

# 쿼리와 유사한 문서를 검색하고 유사도 점수와 함께 반환합니다.
docs_and_scores = db.similarity_search_with_score(query)
#print(docs_and_scores) # 
#contents, scores = docs_and_scores[0]  # 문서와 점수 리스트에서 첫 번째 요소를 선택합니다

# docs_and_scores 은 2개로 이루어진 tuple임.
for c in docs_and_scores:
    content = c[0].page_content
    score = c[1]
    print(content)
    print(score)
    print()


In [ ]:
# 질의를 임베딩 벡터로 변환합니다.
query = "정보보안감사와 관련된 보안정책은?"
embedding_vector = embeddings.embed_query(query)

# 임베딩 벡터를 사용하여 유사도 검색을 수행하고, 문서와 점수를 반환합니다.
docs = db.similarity_search_by_vector(embedding_vector)

# docs 은 1개로 이루어진 tuple임.=> 스코어는 추력 안됨
for c in docs:
    content = c.page_content
    print(content)
    print()


In [ ]:
# 저장과 로딩

# 로컬에 "MY_FIRST_DB_INDEX"라는 이름으로 데이터베이스를 저장합니다.
DB_INDEX = "MY_FIRST_DB_INDEX"
db.save_local(DB_INDEX)

# 로컬에 저장된 데이터베이스를 불러와 new_db 변수에 할당합니다.
new_db = FAISS.load_local(DB_INDEX, embeddings,
                          allow_dangerous_deserialization=True)

query = "정보보안감사와 관련된 보안정책은?"

# new_db에서 query와 유사한 문서를 검색하여 docs 변수에 할당합니다.
docs = new_db.similarity_search(query)

# docs 은 1개로 이루어진 tuple임.=> 스코어는 추력 안됨
for c in docs:
    content = c.page_content
    print(content)
    print()


In [12]:
# 필터링
# FAISS vectorstore는 필터링 기능도 지원할 수 있습니다.
# FAISS는 기본적으로 필터링을 지원하지 않기 때문에, 이를 수동으로 처리해야 합니다.
    
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

# OpenAIEmbeddings를 사용하여 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings()
    
list_of_documents = [
    # 페이지 내용이 "foo"이고 메타데이터로 페이지 번호 1을 가진 문서
    Document(page_content="foo", metadata=dict(page=1)),
    # 페이지 내용이 "bar"이고 메타데이터로 페이지 번호 1을 가진 문서
    Document(page_content="bar", metadata=dict(page=1)),
    # 페이지 내용이 "foo"이고 메타데이터로 페이지 번호 2를 가진 문서
    Document(page_content="foo", metadata=dict(page=2)),
    # 페이지 내용이 "barbar"이고 메타데이터로 페이지 번호 2를 가진 문서
    Document(page_content="barbar", metadata=dict(page=2)),
    # 페이지 내용이 "foo"이고 메타데이터로 페이지 번호 3을 가진 문서
    Document(page_content="foo", metadata=dict(page=3)),
    # 페이지 내용이 "bar burr"이고 메타데이터로 페이지 번호 3을 가진 문서
    Document(page_content="bar burr", metadata=dict(page=3)),
    # 페이지 내용이 "foo"이고 메타데이터로 페이지 번호 4를 가진 문서
    Document(page_content="foo", metadata=dict(page=4)),
    # 페이지 내용이 "bar bruh"이고 메타데이터로 페이지 번호 4를 가진 문서
    Document(page_content="bar bruh", metadata=dict(page=4)),
]
# 문서 리스트와 임베딩을 사용하여 FAISS 데이터베이스 생성
db = FAISS.from_documents(list_of_documents, embeddings)

query = "foo가 속한 말들을 찾아줘"

# "foo"와 유사한 문서를 검색하고 점수와 함께 결과 반환
results_with_scores = db.similarity_search_with_score(query)   
print(results_with_scores)

for doc, score in results_with_scores:  # 검색 결과를 반복하면서
    # 각 문서의 내용, 메타데이터, 점수를 출력
    print(f"Content: {doc.page_content}, Metadata: {doc.metadata}, Score: {score}")

print(f"--"*20)

# 방법1) 유사도 검색을 수행하고 필터를 적용하여 결과와 점수를 반환합니다.
results_with_scores = db.similarity_search_with_score(query, filter=dict(page=1))

# 방법2) 혹은 callable 을 사용하여 필터링 하는 경우
# results_with_scores = db.similarity_search_with_score("foo", filter=lambda d: d["page"] == 1)

for doc, score in results_with_scores:  # 결과와 점수를 반복합니다.
    # 각 문서의 내용, 메타데이터, 점수를 출력합니다.
    print(
        f"[Content] {doc.page_content}, [metadata] {doc.metadata}, [Score] {score}")


[(Document(page_content='foo', metadata={'page': 1}), 0.37699527), (Document(page_content='foo', metadata={'page': 2}), 0.37699527), (Document(page_content='foo', metadata={'page': 3}), 0.37699527), (Document(page_content='foo', metadata={'page': 4}), 0.37699527)]
Content: foo, Metadata: {'page': 1}, Score: 0.3769952654838562
Content: foo, Metadata: {'page': 2}, Score: 0.3769952654838562
Content: foo, Metadata: {'page': 3}, Score: 0.3769952654838562
Content: foo, Metadata: {'page': 4}, Score: 0.3769952654838562
----------------------------------------
[Content] foo, [metadata] {'page': 1}, [Score] 0.3769952654838562
[Content] bar, [metadata] {'page': 1}, [Score] 0.5311874151229858


In [11]:
# MMR 검색 (Maximal Marginal Relevace : 최대 한계 관련성)
#
# 쿼리에 대한 관련 항목을 검색할 때 중복을 피하는 방법 중 하나입니다. 
# 단순히 가장 관련성 높은 항목들만을 검색하는 대신, MMR은 검색된 항목들 사이에 관련성과 다양성 사이의 균형을 보장합니다. 
# 이는 자주 발생할 수 있는, 매우 유사한 항목들만이 검색되는 상황을 방지 하는 데에 유용합니다.
#
# MMR은 두 가지 주요 요소, 즉 쿼리에 대한 문서의 관련성과 이미 선택된 문서들과의 차별성을 동시에 고려 합니다.
# - 첫째로, 쿼리와의 관련성이 높은 문서를 찾는 것이 중요합니다.
# - 둘째로, 이미 선택된 문서들과는 다른 새로운 정보나 관점을 제공하는 문서를 찾는 것입니다.
# 이 두 가지 요소의 균형을 맞추는 것이 MMR의 핵심입니다.

query = "foo가 속한 말들을 찾아줘"
results = db.max_marginal_relevance_search(query, filter=dict(page=1))

for doc in results:
    # 각 문서의 내용과 메타데이터를 출력합니다.
    print(f"[Content] {doc.page_content}, [metadata] {doc.metadata}")
    

[Content] foo, [metadata] {'page': 1}
[Content] bar, [metadata] {'page': 1}


In [13]:
# fetch_k = 후보군 계수 설정
# similarity_search 함수를 호출할 때 fetch_k 매개변수를 설정하는 방법에 대한 예시입니다.
# 일반적으로 fetch_k 매개변수는 k 매개변수보다 훨씬 큰 값으로 설정하는 것이 좋습니다. 
# 그 이유는 fetch_k 매개변수가 필터링 전에 가져올 문서의 수를 나타내기 때문입니다.
# *만약 fetch_k를 낮은 값으로 설정하면, 필터링할 문서가 충분하지 않을 수 있습니다.

query = "foo가 속한 말들을 찾아줘"

results = db.similarity_search(
    query,  # 검색 쿼리
    # 메타데이터의 'page' 필드가 1인 문서만 필터링
    filter=dict(page=1),
    k=1,  # 가장 유사한 1개의 문서를 반환
    fetch_k=4,
)  # 4개의 문서까지 검색

for doc in results:
    # 각 문서의 내용과 메타데이터를 출력합니다.
    print(f"[Content] {doc.page_content}, [metadata] {doc.metadata}")

[Content] foo, [metadata] {'page': 1}
